In [6]:
%pip install yfinance
%pip install pandas
%pip install statsmodels
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
import numpy as np

In [8]:
d=yf.download(["DASH","UBER"], start=start, end=end)

NameError: name 'start' is not defined

In [ ]:
d

In [ ]:

# 1. Download data (adjust dates as you like)
start = "2019-01-01"
end   = "2025-10-31"  # or today
d = yf.download(["V","MA"], start=start, end=end)["Close"].dropna()

# 2. Optionally take log-prices (often used)
d_log = d.apply(lambda x: np.log(x))

# 3. Run cointegration test (Engle-Granger)
coint_stat, pvalue, crit_vals = coint(d_log["V"], d_log["MA"])
print("t-stat:", coint_stat)
print("p-value:", pvalue)
print("critical values:", crit_vals)

# 4. Interpret:
if pvalue < 0.05:
    print("Evidence of cointegration → good candidate for pairs trading")
else:
    print("No statistical evidence of cointegration")

In [ ]:
X = sm.add_constant(d_log["MA"])
model = sm.OLS(d_log["V"], X).fit()
beta = model.params["MA"]

print("Hedge ratio (beta):", beta)

In [ ]:
spread = d_log["V"] - beta * d_log["MA"]

In [ ]:
window = 30  
spread_mean = spread.rolling(window).mean()
spread_std = spread.rolling(window).std()

zscore = (spread - spread_mean) / spread_std

In [ ]:
signals = pd.Series(0, index=zscore.index)

signals[zscore > 1] = -1   # short V, long MA
signals[zscore < -1] = 1   # long V, short MA

In [ ]:
returns = d_log.diff()

In [ ]:
strategy_returns = signals.shift(1) * (
    returns["V"] - beta * returns["MA"]
)

In [ ]:
cumulative_returns = strategy_returns.cumsum()

cumulative_returns.plot(
    title="Pairs Trading Strategy: V–MA",
    ylabel="Returns",
    figsize=(10,5)
)

In [ ]:
signals.value_counts()

In [ ]:
spread.plot(title="V–MA Spread")

In [ ]:
sharpe = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)

In [ ]:
sharpe
